In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pymongo

In [5]:
from pubcrawler.article import Article

In [18]:
from reporter import MongoQueryReporter

In [7]:
client = pymongo.MongoClient("localhost", 27017)
articles = client.pmc.articles

In [8]:
# To take the first one
record = articles.find_one()
article = Article(record["xml"])

In [86]:
# To sample a random article:
record = articles.aggregate([
    { "$sample": { "size": 1 } }
]).next()
article = Article(record["xml"])

In [27]:
# An article with tables:
record = articles.find(
    { "_id": "6112272" }
).next()
article = Article(record["xml"])

In [22]:
print(article.pub_ids())

{'pmid': '30159487', 'pmc': '6112272', 'publisher-id': 'S2468-2659(16)30016-6', 'doi': '10.1016/j.pld.2016.09.005'}


In [28]:
print(record["extracted_text"])

1


Introduction

Loss of continuous natural areas and conversion of forests to smaller areas have drastically reduced the size of plant and animal populations, increasing the isolation between populations of the same species (Young and Clarke, 2000, Lowe et al., 2005). Plant reproduction may decrease in fragmented populations because plants may receive fewer flower visitors due to a decline in the richness and abundance of pollinators, modification in species composition and limitation in movement among landscapes (Young and Clarke, 2000, Cascante et al., 2002, Goverde et al., 2002, Frankham, 2010). Disturbance also changes the abundance, richness or behavior of animal seed dispersers affecting seed dispersal (Ghazoul, 2005, Lowe et al., 2005), recruitment and establishing of seedlings (Ewers and Didham, 2006), and the spatial genetic structure of plant populations (Hamrick, 2004). Shortly following a disturbance, levels of genetic homozygosity in affected plant populations are predic

In [26]:
cursor = articles.find({})
reporter = MongoQueryReporter(1, articles, {})

for idx, record in enumerate(cursor):
    article = Article(record["xml"])

    record_update = {
        "extracted_text": article.extract_text()
    }

    articles.update_one(
        filter={"_id": record["_id"]}, update={"$set": record_update}
    )

    reporter.report(idx)



In [11]:
terms

['field work',
 'fieldwork',
 'field study',
 'field site',
 'field area',
 'study site',
 'study location',
 'study area',
 'research site',
 'research location',
 'sampling site',
 'sampling location',
 'sampling area']

In [11]:
def print_article(article):
    print("""
PMC ID

{}


TITLE

{}


METADATA

{}


TEXT MATCHES

{}


EXTRACTED TEXT

{}""".format(article.get("_id"),
             article.get("article_title"),
             article.get("article_meta"),
             article.get("text_matches"),
             article.get("extracted_text").strip()))

In [12]:
def view_one_article(query={ "text_matches": { "$in": terms } }):
    query = [
        { "$match": query },
        { "$sample": { "size": 1 } }
    ]
    article = articles.aggregate(query).next()
    print("""
PMC ID

{}


TITLE

{}


METADATA

{}


TEXT MATCHES

{}


EXTRACTED TEXT

{}""".format(article.get("_id"),
             article.get("article_title"),
             article.get("article_meta"),
             article.get("text_matches"),
             article.get("extracted_text").strip()))

In [13]:
terms

['field work',
 'fieldwork',
 'field study',
 'field site',
 'field area',
 'study site',
 'study location',
 'study area',
 'research site',
 'research location',
 'sampling site',
 'sampling location',
 'sampling area']

In [18]:
view_one_article()


PMC ID

2824121


TITLE

Phase II study of motesanib in Japanese patients with advanced gastrointestinal stromal tumors with prior exposure to imatinib mesylate


METADATA

{'article_type': 'research-article', 'has_body': True, 'all_geospans': 15, 'geospan_density': 0.0007973633850733575, 'n_geospans': 15}


TEXT MATCHES

['study site']


EXTRACTED TEXT

Introduction

Gastrointestinal stromal tumor (GIST) is a rare stromal neoplasm that predominantly arises from the muscularis propria layers, representing the most common mesenchymal tumor of the gastrointestinal tract. Although the primary therapy for nonmetastatic GIST is surgical resection, there still remain unresectable cases of advanced or metastatic GIST. Unresectable GISTs are resistant to conventional chemotherapy and radiotherapy [1]. Before imatinib mesylate was introduced in clinical practice, the prognosis for patients with unresectable GIST was dismal, with a median survival period of 22 months [2].

The critical transfor

## View Articles at Different GeoName Densities

In [13]:
query = {
    "text_matches": {"$in": terms},
    "article_meta.article_type": "research-article",
    "article_meta.has_body": True,
    "article_meta.n_geospans": {"$exists": True}
}

cursor = articles.aggregate([
    {"$match": query},
    {"$project": {
        "n_geospans": "$article_meta.n_geospans",
        "length": {"$strLenCP": "$extracted_text"}
    }}
])

subset_df = pd.DataFrame(cursor)

In [81]:
deciles = subset_df.quantile(q=np.arange(0, 1.1, 0.1))
quantiles = subset_df.quantile(q=[0.05, 0.25, 0.5, 0.75, 0.95])

In [82]:
deciles

,length,n_geospans
0.0,3048.0,0.0
0.1,18614.0,7.0
0.2,22789.0,11.0
0.3,25835.0,16.0
0.4,28323.0,21.0
0.5,31158.0,26.0
0.6,34320.0,34.0
0.7,37611.0,43.0
0.8,42225.0,58.0
0.9,50307.0,90.0


In [83]:
quantiles

,length,n_geospans
0.05,15334.0,4.0
0.25,24279.0,14.0
0.50,31158.0,26.0
0.75,39655.0,50.0
0.95,56676.0,128.0


In [97]:
quantiles = subset_df.quantile(q=[0.01, 0.25, 0.95, 0.99])
quantiles

,length,n_geospans
0.01,10097.2,1.0
0.25,24279.0,14.0
0.95,56676.0,128.0
0.99,77687.0,234.2


In [47]:
lower_break = 9
upper_break = 10

In [84]:
quantiles.n_geospans.iloc[4]

128.0

In [85]:
geospan_query = {
    "text_matches": {"$in": terms},
    "article_meta.article_type": "research-article",
    "article_meta.has_body": True,
    "article_meta.n_geospans": {"$gte": quantiles.n_geospans.iloc[4],
                                "$lte": deciles.n_geospans.iloc[upper_break], }
}

In [95]:
view_one_article(geospan_query)


PMC ID

5678892


TITLE

Phylogeny of xerophilic aspergilli (subgenus Aspergillus) and taxonomic revision of section Restricti


METADATA

{'all_geospans': 695, 'n_geospans': 695, 'article_type': 'research-article', 'has_body': True}


TEXT MATCHES

['sampling site']


EXTRACTED TEXT

Introduction


Aspergillus section Restricti species occurs in environments with low water activity (aw). They are commonly found on building materials, in house dust (Kaarakainen et al., 2009, Visagie et al., 2014) and dried, salted or high sugar content foods (Pitt and Hocking, 2009, Frasz and Miller, 2015). Much attention is being paid to the indoor air quality (Kasuga, 2012, Flannigan et al., 2016) and these fungi are repeatedly reported to be present in this environment (Samson et al., 2002, Meklin et al., 2004, Meklin et al., 2007), where they are considered a potential agent responsible for sick building syndrome, respiratory problems and allergies (Terr, 2009, Saijo et al., 2011, Abe, 2012). Spec

In [51]:
length_query = {
    "text_matches": {"$in": terms},
    "article_meta.article_type": "research-article",
    "article_meta.has_body": True,
    "article_meta.n_geospans": {"$exists": True},
    "$expr": {"$gte": [{"$strLenCP": "$extracted_text"}, quantiles.length.iloc[lower_break]]},
    "$expr": {"$lte": [{"$strLenCP": "$extracted_text"}, quantiles.length.iloc[upper_break]]}
}

In [55]:
view_one_article(length_query)


PMC ID

6132394


TITLE

Assessment of seafood contamination under the marine strategy framework directive: contributions of the German environmental specimen bank


METADATA

{'all_geospans': 113, 'n_geospans': 113, 'article_type': 'research-article', 'has_body': True}


TEXT MATCHES

['sampling site', 'sampling area']


EXTRACTED TEXT

Introduction

Marine environments are under massive pressure caused by anthropogenic exploitation and pollution. This includes pollution with chemical substances and marine litter, extensive fishing activities, deterioration of the sea floor, e.g., by construction measures, extraction of minerals, and fishing with ground nets, and introduction of noise, e.g., by ships, construction, renewable energy, and tourism (Ban and Alder 2008; Boldt et al. 2014; Halpern et al. 2008; IOC/UNESCO et al. 2011). As a consequence, the biodiversity has decreased, fish stocks have declined, and the resilience of marine ecosystems has dwindled in many parts of the world 

In [108]:
quantiles = subset_df.quantile(q=[0.01, 0.25, 0.95, 0.99])

In [109]:
quantiles

,length,n_geospans
0.01,10097.2,1.0
0.25,24279.0,14.0
0.95,56676.0,128.0
0.99,77687.0,234.2


In [134]:
simple_query = {
    "text_matches": {"$in": terms},
    "article_meta.article_type": "research-article",
    "article_meta.has_body": True,
    "article_meta.n_geospans": {"$exists": True},
    "article_meta.geospan_density": {"$exists": True},
}

cursor = articles.aggregate([
    {"$match": simple_query},
    {"$project": {
        "n_geospans": "$article_meta.n_geospans",
        "geospan_density": "$article_meta.geospan_density",
        "length": {"$strLenCP": "$extracted_text"}
    }}
])

subset_df = pd.DataFrame(cursor)
quantiles = subset_df.quantile(q=[0.01, 0.25, 0.95, 0.99])

In [158]:
density = subset_df["geospan_density"]
density = density[density!=0]
log_density = np.log(density)
threshold_density = np.exp(np.mean(log_density) - np.std(log_density))

threshold_density

0.00013150962697391707

In [157]:
density[density>threshold_density]

0    0.000193
1    0.000772
2    0.000238
5    0.001409
6    0.000143
7    0.000825
8    0.000601
Name: geospan_density, dtype: float64

In [142]:
import numpy as np

In [146]:
np.log(density)

/Users/toph/.virtualenvs/ai4e-data-labeling/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


-inf

In [141]:
log_density = log(density)

NameError: name 'log' is not defined

In [105]:
final_query = {
    "text_matches": {"$in": terms},
    "article_meta.article_type": "research-article",
    "article_meta.has_body": True,
    "article_meta.n_geospans": {"$gt": quantiles.n_geospans.iloc[1],
                                "$lt": quantiles.n_geospans.iloc[3]},
    "$expr": {"$gte": [{"$strLenCP": "$extracted_text"}, quantiles.length.iloc[0]]},
    "$expr": {"$lte": [{"$strLenCP": "$extracted_text"}, quantiles.length.iloc[2]]}
}

In [112]:
final_query = {
    "text_matches": {"$in": terms},
    "article_meta.article_type": "research-article",
    "article_meta.has_body": True,
    "article_meta.n_geospans": {"$gt": quantiles.n_geospans.iloc[1],
                                "$lte": quantiles.n_geospans.iloc[3]},
    "$and": [
        {"$expr": {"$gt": [{"$strLenCP": "$extracted_text"}, quantiles.length.iloc[0]]}},
        {"$expr": {"$lte": [{"$strLenCP": "$extracted_text"}, quantiles.length.iloc[2]]}}
    ]
}

In [113]:
final_query

{'text_matches': {'$in': ['field work',
   'fieldwork',
   'field study',
   'field site',
   'field area',
   'study site',
   'study location',
   'study area',
   'research site',
   'research location',
   'sampling site',
   'sampling location',
   'sampling area']},
 'article_meta.article_type': 'research-article',
 'article_meta.has_body': True,
 'article_meta.n_geospans': {'$gt': 14.0, '$lte': 234.20000000000027},
 '$and': [{'$expr': {'$gt': [{'$strLenCP': '$extracted_text'}, 10097.2]}},
  {'$expr': {'$lte': [{'$strLenCP': '$extracted_text'}, 56676.0]}}]}

In [125]:
view_one_article(final_query)


PMC ID

4829693


TITLE

Groundwater–surface water mixing shifts ecological assembly processes and stimulates organic carbon turnover


METADATA

{'all_geospans': 29, 'n_geospans': 29, 'article_type': 'research-article', 'has_body': True}


TEXT MATCHES

['sampling location']


EXTRACTED TEXT

Understanding the processes that underlie places and times of elevated biogeochemical activity is needed to construct predictive models of elemental cycling and contaminant fate and transport under future environmental scenarios. Without the knowledge of governing processes, predictions are unlikely to be accurate, owing to emergent properties that arise through the behaviour of complex systems1. Environmental transition zones often exhibit elevated rates of biogeochemical activity due to the mixing of complementary electron donors and terminal electron acceptors234. The hyporheic zone is a classic transition zone567 important to the functioning of lotic systems891011121314, where biogeochemical

In [128]:
xml = articles.find_one({"_id": "4546277"})["extracted_text"]

In [130]:
len(xml)

30029

In [66]:
soup = BeautifulSoup(xml)

In [69]:
print(soup.prettify())

<!DOCTYPE article PUBLIC "-//NLM//DTD JATS (Z39.96) Journal Archiving and Interchange DTD v1.0 20120330//EN" "JATS-archivearticle1.dtd">
<html>
 <body>
  <article article-type="research-article" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink">
   <?properties open_access?>
   <front>
    <journal-meta>
     <journal-id journal-id-type="nlm-ta">
      PLoS One
     </journal-id>
     <journal-id journal-id-type="iso-abbrev">
      PLoS ONE
     </journal-id>
     <journal-id journal-id-type="publisher-id">
      plos
     </journal-id>
     <journal-id journal-id-type="pmc">
      plosone
     </journal-id>
     <journal-title-group>
      <journal-title>
       PLoS ONE
      </journal-title>
     </journal-title-group>
     <issn pub-type="epub">
      1932-6203
     </issn>
     <publisher>
      <publisher-name>
       Public Library of Science
      </publisher-name>
      <publisher-loc>
       San Francisco, CA USA
      </publisher-loc>


In [73]:
art = Article(xml)

In [75]:
print(art.extract_text())




Introduction

Air pollution is a problem for much of the developing world and is believed to kill more people worldwide than AIDS, malaria, breast cancer, or tuberculosis [1–4]. Airborne particulate matter (PM) is especially detrimental to health [5–8], and has previously been estimated to cause between 3 and 7 million deaths every year, primarily by creating or worsening cardiorespiratory disease [2–4,6,7]. Particulate sources include electric power plants, industrial facilities, automobiles, biomass burning, and fossil fuels used in homes and factories for heating. In China, air pollution was previously estimated to contribute to 1.2 to 2 million deaths annually [2–4].

In 2012, China adopted the Ambient Air Quality Standard [9], and began development of a national Air Reporting System that now includes 945 sites in 190 cities. These automated stations report hourly via the internet, and focus on six pollutants: particulate matter < 2.5 microns (PM2.5), particulate matter < 10 mic